# What is the doggest dog?

This is a notebook that can be used to perform experiments connected to our paper.

The aim of our method is to determine prototypes and anti-prototypes for a given basic-level category. Our methods aim to examine deep learning models from the perspective of the Prototype Theory.

Method from the paper: Strategy 3 (based on Multidimensional Scaling)

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

We use the following function to find the hyponyms for a given basic-level category - hypernym:

In [2]:
def find_hyponyms(hypernym_name):
    """
    Function ca be used to obtain a list of indexes of the ImageNet classes 
    belonging to a given basic-level category (a hypernym).
    ------
    Pramaters: 
    
    hypernym_name: a desired hypernym (e.g. "domestic_cat")
    """
    from nltk.corpus import wordnet as wn
    import nltk
    from tensorflow.keras.applications.imagenet_utils import decode_predictions
    from tensorflow.keras.utils import to_categorical
    import numpy as np
    
    hyponyms = wn.synsets(hypernym_name)[0]
    hyponyms = set([i for i in hyponyms.closure(lambda s:s.hyponyms())])
    offsets = []
    imagenet_classes = decode_predictions(to_categorical(np.expand_dims(np.array(range(1000)), axis=-1), num_classes=1000), top=1)

    for c in imagenet_classes:
        offsets.append(int(c[0][0].split('n')[1]))
    
    ids = []
    for idx, o in enumerate(offsets):
        isadoggo = wn.synset_from_pos_and_offset('n', int(o))
        if isadoggo in hyponyms:
            ids.append(idx)
    return np.array(ids)

In [3]:
def determine_prototype_antiprototype_with_MDS(weights, hyponym_classes, random_seed=None):
    """
    Function can be used to find prototypes and anti-prototypes for a given basic-level category 
    for a given deep learning model. It is a single iteration of MDS.
    ------
    Parameters: 
    
    weights: extracted weights of categories of interest (+ a central element obtained as 
    an element-wise avg of all weights)
    hyponym_classes: indexes of hyponyms (ImageNet classes to be embedded).
    random_state: number used for MDS initialization (of points in MDS' 2D space).
    """
    from sklearn.manifold import MDS
    from sklearn.metrics.pairwise import cosine_distances
    from sklearn.metrics.pairwise import euclidean_distances
    from tensorflow.keras.applications.imagenet_utils import decode_predictions
    from tensorflow.keras.utils import to_categorical
    
    cosine_distances = cosine_distances(weights)
    if random_seed is not None:
        embedding = MDS(n_components=2, normalized_stress='auto', dissimilarity='precomputed', random_state=random_seed)
    else:
        embedding = MDS(n_components=2, normalized_stress='auto', dissimilarity='precomputed')
    X_transformed = embedding.fit_transform(cosine_distances)
    idx = hyponym_classes[np.argmin(euclidean_distances(np.expand_dims(X_transformed[-1], axis=0), X_transformed[:-2]))]
    idx_distant = hyponym_classes[np.argmax(euclidean_distances(np.expand_dims(X_transformed[-1], axis=0), X_transformed[:-2]))]
    prototype = decode_predictions(to_categorical([[idx]], num_classes=1000), top=1)[0][0][1]
    antiprototype = decode_predictions(to_categorical([[idx_distant]], num_classes=1000), top=1)[0][0][1]
    return prototype, antiprototype

In [4]:
def return_prototype_antiprototype(model, hypernym_name, keras=True, levit=False, N_MDS=500, seeds=None):
    """
    Function can be used to find prototypes and anti-prototypes for a given basic-level category 
    for a given deep learning model. 
    Parameters: 
    model: keras/torch model trained on ImageNet (with a standard order of classes that can be decoded with 
    tensorflow.keras.applications.imagenet_utils.decode_predictions
    hypernym_name: string with a name of a basic-level category, e.g. "domestic_cat", "dog" etc.
    high_level_category: in strategy 2, besides the hyponyms of our desired category, we also use
    a contrasting category. A contrasting category is a complement of a set of some higher-level categories. 
    keras: flag whether we want to use a keras model (set True if yes - it is a default). 
    Set false in the case of using a torch model.
    levit: set with keras=False in the case of using one of the LeViTs models.
    N_MDS: number of MDS algorithms runs to find a prototype and anti-prototype
    seeds: a numpy array of seeds (with length N_MDS) to generate 
    the results (use numpy.random.randint(0, 5000, size=500) 
    with numpy.random.seed(55) for the reproduction of results.
    Alternatively, one can used the already generated list of numbers
    from file random_nums.txt (generated this way). 
    """
    from tensorflow.keras.applications.imagenet_utils import decode_predictions
    from sklearn.metrics.pairwise import cosine_similarity
    from tensorflow.keras.utils import to_categorical

    
    hyponym_classes = find_hyponyms(hypernym_name=hypernym_name)    
    print(f"Members of the basic-level category (for hypernym = {hypernym_name}):")
    print(len(hyponym_classes))
    
    # Extraction of weights (we consider models from Keras Application (https://keras.io/api/applications/)
    # and HuggingFace (https://huggingface.co/models) listed in the provided file Model_List.pdf
    if keras:
        hyponym_weights = np.moveaxis(model.layers[-1].get_weights()[0][:, hyponym_classes], -1, 0)
        hypernym_weights = np.mean(model.layers[-1].get_weights()[0][:, hyponym_classes], axis=1)
        hypernym_weights = np.expand_dims(hypernym_weights, axis=0)
        all_weights = np.concatenate([hyponym_weights, hypernym_weights], axis=0)
    else:
        if not levit:
            hyponym_weights = model.classifier.weight.detach().numpy()[hyponym_classes, :]
            hypernym_weights = np.mean(hyponym_weights, axis=0)
            hypernym_weights = np.expand_dims(hypernym_weights, axis=0)
            all_weights = np.concatenate([hyponym_weights, hypernym_weights], axis=0)
        else:
            # models from the LeViT family have slightly different model structure than the other models 
            # from HuggingFace
            hyponym_weights = model.classifier_distill.linear.weight.detach().numpy()[hyponym_classes, :]
            hypernym_weights = np.mean(hyponym_weights, axis=0)
            hypernym_weights = np.expand_dims(hypernym_weights, axis=0)
            all_weights = np.concatenate([hyponym_weights, hypernym_weights], axis=0)
            
    # Running MDS for 500 times to find prototypes and anti-types

    potential_prototypes = []
    potential_antiprototypes = []
    for i in range(N_MDS):
        if seeds is not None and len(seeds) == (N_MDS):
            maxx, minn = determine_prototype_antiprototype_with_MDS(all_weights, hyponym_classes, seeds[i])
        else:
            maxx, minn = determine_prototype_antiprototype_with_MDS(all_weights, hyponym_classes)
        potential_prototypes.append(maxx)
        potential_antiprototypes.append(minn)
        
    prototype = max(set(potential_prototypes), key = potential_prototypes.count)
    antiprototype = max(set(potential_antiprototypes), key = potential_antiprototypes.count)
    print("prototype")
    print(prototype)
    print("anti-prototype")
    print(antiprototype)
    return prototype, antiprototype 

Below one needs to set a WordNet node name - a desired basic-level category.

**In our experiments, we use the following nodes:**

Natural categories:

* *domestic_cat*
* *dog*
* *bird*
* *fish*
* *mammal*

Artificial category: *musical_instrument*

In [5]:
hypernym_name = "domestic_cat"

In [6]:
# example (in our experiments for the purpose of our paper, we use N_MDS=500, and seeds generated
# via numpy.random.randint(0, 5000, size=500) with numpy.random.seed(55)
N_MDS = 3
seeds = np.array([4557, 4762, 4391])

Example for keras:

In [7]:
# generating results for different CNNs (keras)

# ---------------------------------------------------------------------
model = tf.keras.applications.InceptionV3(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, N_MDS=N_MDS, seeds=seeds,  
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

inception_v3
Members of the basic-level category (for hypernym = domestic_cat):
5
prototype
tabby
anti-prototype
Siamese_cat


Example for torch models:

In [9]:
# generating results for different transformers (pytorch) - standard + example LeViT

from transformers import AutoModelForImageClassification

name = "microsoft/swinv2-tiny-patch4-window8-256"
model = AutoModelForImageClassification.from_pretrained(name)

print(name)
most_similar, least_similar = return_prototype_antiprototype(model=model, N_MDS=N_MDS, seeds=seeds,  
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

# ---------------------------------------------------------------

name = 'facebook/levit-384'
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, N_MDS=N_MDS, seeds=seeds,  
                                                        hypernym_name=hypernym_name, 
                                                        keras=False, levit=True)

microsoft/swinv2-tiny-patch4-window8-256
Members of the basic-level category (for hypernym = domestic_cat):
5
prototype
tabby
anti-prototype
Siamese_cat
LevitForImageClassificationWithTeacher
Members of the basic-level category (for hypernym = domestic_cat):
5
prototype
tabby
anti-prototype
Siamese_cat
